# Binary CSP solver

In [1]:
mutable struct CSP
    n_variables::Int64
    domain::Array{Array{Int64}, 1}
    possible_value::Array{Bool, 2}
    
    max_domain::Int64
    
    n_constraints::Int64
    constraint_var1::Array{Int64, 1}
    constraint_var2::Array{Int64, 1}
    constraint_satisfaction::Array{Array{Bool, 2}, 1}
    
    var1_constraints::Array{Array{Int64, 1}, 1}
    var2_constraints::Array{Array{Int64, 1}, 1}
    
    CSP() = new(
        0, [], Array{Bool, 2}(undef, 0, 0),
        0,
        0, [], [], [],
        [], [],
    )
end

In [2]:
function add_variable!(csp::CSP, domain::Array{Int64})
    csp.n_variables += 1
    push!(csp.domain, domain)
    push!(csp.var1_constraints, Array{Int64, 1}(undef, 0))
    push!(csp.var2_constraints, Array{Int64, 1}(undef, 0))
    csp.max_domain = max(csp.max_domain, length(domain))
end

add_variable! (generic function with 1 method)

In [3]:
function build_possible_values!(csp::CSP)
    csp.possible_value::Array{Bool, 2} = zeros(Bool, csp.n_variables, csp.max_domain)
    for var in 1:csp.n_variables
        for value in csp.domain[var]
            csp.possible_value[var, value] = true
        end
    end
end

build_possible_values! (generic function with 1 method)

In [4]:
function add_constraint!(csp::CSP, var1::Int64, var2::Int64, compatibility::Function)
    csp.n_constraints += 1

    m1::Int64 = length(csp.domain[var1])
    m2::Int64 = length(csp.domain[var2])
    satisfaction::Array{Bool, 2} = zeros(Int64, m1, m2)
    for value1 in csp.domain[var1], value2 in csp.domain[var2]
        satisfaction[value1, value2] = compatibility(var1, var2, value1, value2)
    end
    
    push!(csp.constraint_var1, var1)
    push!(csp.constraint_var2, var2)
    push!(csp.constraint_satisfaction, satisfaction)
    
    push!(csp.var1_constraints[var1], csp.n_constraints)
    push!(csp.var2_constraints[var2], csp.n_constraints)
end

add_constraint! (generic function with 1 method)

In [5]:
function check_feasibility(csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64)
    if new_var == 0
        return true
    end
    for cons in csp.var1_constraints[new_var]
        var2::Int64 = csp.constraint_var2[cons]
        satisfaction = csp.constraint_satisfaction[cons]
        value1 = instantiation[new_var]
        value2 = instantiation[var2]
        if (value1 !== 0) && (value2 !== 0)
            if !satisfaction[value1, value2]
                return false
            end
        end
    end
    return true
end

check_feasibility (generic function with 1 method)

In [6]:
function discard_values!(csp::CSP, to_discard::Dict{Int64, Array{Int64}})
    for (var2, discarded_var2) in to_discard
        for value2 in discarded_var2
            csp.possible_value[var2, value2] = false
        end
    end
end

function undo_discard_values!(csp::CSP, to_discard::Dict{Int64, Array{Int64}})
    for (var2, discarded_var2) in to_discard
        for value2 in discarded_var2
            csp.possible_value[var2, value2] = true
        end
    end
end

undo_discard_values! (generic function with 1 method)

In [7]:
function forward_checking(csp::CSP, instantiation::Array{Int64, 1}, x::Int64)
    to_discard::Dict{Int64, Array{Int64}} = Dict{Int64, Array{Int64}}()
    
    a = instantiation[x]
    to_discard[x] = []
    for aa in csp.domain[x]
        if (aa != a) && csp.possible_value[x, aa]
            push!(to_discard[x], aa)
        end
    end
    
    for cons in csp.var1_constraints[x]
        y = csp.constraint_var2[cons]
        if instantiation[y] != 0
            continue
        end
        
        to_discard[y] = []
        for b in csp.domain[y]
            if (!csp.constraint_satisfaction[cons][a, b]) && csp.possible_value[y, b]
                push!(to_discard[y], b)
            end
        end
    end
    return to_discard
end

forward_checking (generic function with 1 method)

In [8]:
function choose_next_variable(csp::CSP, instantiation::Array{Int64, 1})
    domain_size::Array{Int64, 1} = sum(csp.possible_value, dims=2)[:, 1]
    best_variable::Int64 = 0
    min_domain::Int64 = typemax(Int64)
    for var in 1:csp.n_variables
        if instantiation[var] != 0
            continue
        elseif domain_size[var] < min_domain
            best_variable = var
            min_domain = domain_size[var]
        end
    end
    return best_variable
end

choose_next_variable (generic function with 1 method)

In [13]:
function backtrack!(csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64, nodes_explored::Int64)
    if !check_feasibility(csp, instantiation, new_var)
        return (false, instantiation, nodes_explored)
    end
    
    new_var = choose_next_variable(csp, instantiation)
    if new_var == 0
        return true, instantiation, nodes_explored
    end
        
    for new_value in csp.domain[new_var]
                
        if !csp.possible_value[new_var, new_value]
            continue
        end

        nodes_explored += 1
        instantiation[new_var] = new_value
        
        to_discard = forward_checking(csp, instantiation, new_var)
        discard_values!(csp, to_discard)
                
        (solution_found, solution, nodes_explored) = backtrack!(csp, instantiation, new_var, nodes_explored)
        if solution_found
            return (true, solution, nodes_explored)
        end
        
        instantiation[new_var] = 0
        undo_discard_values!(csp, to_discard)

    end
    
    return false, instantiation, nodes_explored
end

function backtrack!(csp::CSP)
    return backtrack!(csp, zeros(Int64, csp.n_variables), 0, 0)
end

backtrack! (generic function with 2 methods)

In [14]:
function compatibility_nqueens(var1::Int64, var2::Int64, value1::Int64, value2::Int64)
    if value1 == value2
        return false
    elseif (value1 - value2) == (var1 - var2)
        return false
    elseif (value1 - value2) == -(var1 - var2)
        return false
    else
        return true
    end
end

function define_nqueens(n::Int64)
    csp = CSP()
    domain = collect(1:n)
    for i in 1:n
        add_variable!(csp, domain)
    end
    build_possible_values!(csp)
    for i in 1:n, j in 1:n
        if (i != j)
            add_constraint!(csp, i, j, compatibility_nqueens)
        end
    end
    return csp
end

function visualize_nqueens(instantiation::Array{Int64})
    n::Int64 = length(instantiation)
    for i in 1:n
        println()
        for j in 1:n
            if instantiation[i] == j
                print(" o")
            else
                print(" .")
            end
        end
    end
end        

visualize_nqueens (generic function with 1 method)

In [15]:
@time nqueens = define_nqueens(100)
@time solution_exists, solution, nodes_explored = backtrack!(nqueens)
print("Nodes explored: ", nodes_explored, " - Solution: ", solution)

  1.828192 seconds (90.97 k allocations: 856.206 MiB, 7.17% gc time)
  0.432079 seconds (635.01 k allocations: 62.024 MiB, 1.07% gc time)
Nodes explored: 13550 - Solution: [1, 3, 5, 57, 59, 4, 64, 7, 58, 71, 81, 60, 6, 91, 82, 90, 8, 83, 77, 65, 73, 26, 9, 45, 37, 63, 66, 62, 44, 10, 48, 54, 43, 69, 42, 47, 18, 11, 72, 68, 50, 56, 61, 36, 33, 17, 12, 51, 100, 93, 97, 88, 35, 84, 78, 19, 13, 99, 67, 76, 92, 75, 87, 96, 94, 85, 20, 14, 95, 32, 98, 55, 40, 80, 49, 52, 46, 53, 21, 15, 41, 2, 27, 34, 22, 70, 74, 29, 25, 30, 38, 86, 16, 79, 24, 39, 28, 23, 31, 89]